In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import dask.dataframe as dd
import re
from dask.diagnostics import ProgressBar
import dask

**Step One**
After importing the packages follow these steps to filter the initial dataset from: 
https://data.seattle.gov/Community/Checkouts-by-Title/tmmm-ytt6/data

To just the years 2018 - Present.

**Note**: Dask is used throughout this Notebook instead of Pandas due to the size of the dataset and the limitations of my computer. If your computer is able to handle large data calculations you do not need to repeatedly save different versions of the data. You can simply perform the cleaning steps.

In [ ]:
#seattle_df = dd.read_csv('..\data\checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [ ]:
#filtered_seattle_df = seattle_df[seattle_df['CheckoutYear'] >= 2018]

In [ ]:
#filtered_seattle_df.head()

In [ ]:
#result = filtered_seattle_df.compute()

In [ ]:
#result.to_csv('..\\data\\filtered_checkouts_by_title.csv', index=False)

**Step Two** After performing step one you can load the newly filtered data and review the data following the steps below.

In [ ]:
#seattle_df = dd.read_csv('..\data\\filtered_checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [ ]:
#seattle_df.head()

In [ ]:
#seattle_df.info()

In [ ]:
#result = seattle_df.compute()

In [ ]:
#result.info()

**Step Three** Cleaning the ISBN column by splitting it into the ISBN-10 and the ISBN-13 column as the current data contains both ISBN numbers as comma separated values in the same column.

Finding the corresponding lengths to sort into the correct columns.

Trying for loop and chunk iterator.

In [ ]:
def process_chunk(chunk):
    chunk_copy = chunk.copy()
    chunk_copy['ISBN'] = chunk_copy['ISBN'].astype(str)
    split_isbn = chunk_copy['ISBN'].str.split(',', expand=True)
    chunk_copy.loc[:, 'Possible ISBN-10'] = split_isbn[0].str.strip()
    chunk_copy.loc[:, 'Possible ISBN-13'] = None
    if split_isbn.shape[1] > 1:
        chunk_copy.loc[:, 'Possible ISBN-13'] = split_isbn[1].str.strip()
    chunk_copy.loc[:, 'ISBN-10'] = chunk_copy['Possible ISBN-10'].where(chunk_copy['Possible ISBN-10'].str.len() == 10, np.nan)
    chunk_copy.loc[:, 'ISBN-13'] = chunk_copy['Possible ISBN-13'].where(chunk_copy['Possible ISBN-13'].str.len() == 13, chunk_copy['Possible ISBN-10'])
    chunk_copy = chunk_copy.drop(columns=['Possible ISBN-10', 'Possible ISBN-13'])
    return chunk_copy

In [ ]:
chunk_size = 10**5
chunks = [process_chunk(result[i:i+chunk_size]) for i in range(0, result.shape[0], chunk_size)]

In [ ]:
result_processed = pd.concat(chunks, ignore_index=True)

In [ ]:
result_processed.info()

In [ ]:
result_processed.head()

**Optional** This step is just to check if one or the other has complete data which they do not. Both have null values.

In [ ]:
result_processed['ISBN-13'].isna().sum()

In [ ]:
result_processed['ISBN-10'].isna().sum()

In [ ]:
result_processed['PublicationYear'].isna().sum()

In [ ]:
result_processed.head(5)

**Step Four** The following steps are for cleaning the publication year column. You can skip to step **4.5** if you want to use the cumulative function that performs all of the steps below at once.

**Function One** Leaves null values as null values. Converts year to string. Keeps only digit values that are in year for cases like [1990] and finally checks if the length of the result is 4 digits.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return year 
    year = str(year) 
    cleaned_year = ''.join(filter(str.isdigit, year))
    return cleaned_year[:4] if len(cleaned_year) >= 4 else None

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'].unique()

**Function Two** Leaves null values as nan values. Converts year to string. Similar to function one but attempts to clean better by using a lamda function instead to capture the years better. Then it taked the last four digits and uses that as the year and checks if the result is 4 characters otherwise it will leave it as nan.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    year = str(year)
    year = ''.join(filter(lambda x: x.isdigit(), year))
    year = year[-4:]
    return year if len(year) == 4 else np.nan

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

**Optional** Checking if all the years are clean or if there are still outliers.

In [ ]:
result_processed['Cleaned_Year'].unique()

In [ ]:
result_processed[result_processed['Cleaned_Year'] == '92009']

**Function Three** Checks if the year is a string. Uses regex to find years in the 1900-1999 format. If so it takes the first four digit year group. then returns it.

In [ ]:
def clean_year(year):
    if isinstance(year, str):
        match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
        if match:
            return match.group(1)  
        
        match = re.match(r"(\d{4})(\d{4})", year)
        if match:
            return match.group(1)
        
    return year

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

**Step 4.2** First half of manual cleaning.

In [ ]:
result_processed['Cleaned_Year'].unique()

In [ ]:
result_processed[result_processed['Cleaned_Year'] == 2022.1]

**Step 4.2** Second half of manual cleaning of outliers. Look at specific rows to fix by identifying outliers in the steps from the first half then putting the unique index in the code below and updating it accordingly. Ex. 2022.1 -> Index 12303238 -> 2022. After that replace it using the .loc lines for both the 'PublicationYear' and the 'Cleaned_Year' columns.

In [ ]:
indexes_to_update = [12303238]

In [ ]:
new_year = 2022

In [ ]:
result_processed.loc[indexes_to_update, 'PublicationYear'] = new_year

In [ ]:
result_processed.loc[indexes_to_update, 'Cleaned_Year'] = new_year

**Step 4.5** Instead of running the three functions from step 4 individually you can run this function which combines their actions into one larger function to achieve the same effect.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    
    year = str(year)
    match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
    if match:
        return match.group(1)
    
    match = re.match(r"(\d{4})(\d{4})", year)
    if match:
        return match.group(1)
    
    year = ''.join(filter(lambda x: x.isdigit(), year))
    return year[-4:] if len(year) == 4 else np.nan

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

In [ ]:
result_processed['Cleaned_Year'].unique()

**Optional** Change the 'Cleaned_Year' column back into Int64 data type.

In [ ]:
result_processed['Cleaned_Year'] = result_processed['Cleaned_Year'].astype('Int64')

In [ ]:
result_processed.head(20)

**Optional** Save this data to a new CSV file.

In [ ]:
result_processed.to_csv('cleaned_checkouts_data.csv', index=False)

**Step Five** Load the new file or move on to the next portion of data cleaning if you skipped the multiple csv file saves.

In [ ]:
dtypes = {
    'ISBN': 'object',
    'ISBN-10': 'object',
    'ISBN-13': 'object'
}
seattle_df = dd.read_csv('..\data\cleaned_checkouts_data.csv', dtype=dtypes)

In [ ]:
result = seattle_df.compute()

In [ ]:
result.head(20)

**Optional** Look at the unique subjects as the next cleaning step will be to attempt to clean the 'Subjects' column.

In [ ]:
result['Subjects'].nunique()

**Optional** The following two pieces of code are to toggle the display limits so that you can see any number of rows you like when running the value counts code. 

**Note** If you do not toggle it back off you run a risk of crashing the kernel if you try to look at too many rows.

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
#pd.reset_option('display.max_rows')

**Optional** Useful to get an idea of what the subjects look like and what still needs to be cleaned.

In [ ]:
result['Subjects'].value_counts().head(200)

In [ ]:
result['Subjects'].value_counts().tail(50)

**Step Six** Function to clean based on CSV value separations on the 'Subject' column. It will take the first two values in a comma separated list and put them into a pair that can then be split into new columns.

**Note** Any time you need to rerun this you need to start from the data loading step or it will not run correctly.

In [ ]:
def clean_subject(subject):
    if isinstance(subject, str):
        keywords = [word.strip() for word in subject.split(',')]
        primary_category = keywords[0] if keywords else 'Uncategorized'
        secondary_category = keywords[1] if len(keywords) > 1 else 'General'
        return primary_category, secondary_category
    return 'Uncategorized', 'Uncategorized'

In [ ]:
ddf = dd.from_pandas(result, npartitions=8) 

In [ ]:
ddf['Cleaned_Categories'] = ddf['Subjects'].apply(clean_subject, meta=('x', 'f8'))

**Optional** Troubleshooting step in case there are issues with the function from step 6.

In [ ]:
#print(ddf['Cleaned_Categories'].compute().unique())

**Step Six** Function that extracts the categories from the previous steps, verifies the length, and if anything fails will return "none".

In [1]:
def extract_category(categories, position):
    if categories and isinstance(categories, tuple) and len(categories) > position:
        return categories[position]
    else:
        return None

**Step Seven** Code that assigns the subjects to new columns. The first value in the comma separated list is assigned to the 'Cleaned_Primary' column and the second value is assigned to the "Cleaned_Secondary' column.

**Note** Dask will not run if you try to combine these into one line of code. Each column has to be handled one at a time.

In [ ]:
ddf['Cleaned_Primary'] = ddf['Cleaned_Categories'].apply(lambda x: extract_category(x, 0), meta=('x', 'O'))
ddf['Cleaned_Secondary'] = ddf['Cleaned_Categories'].apply(lambda x: extract_category(x, 1), meta=('x', 'O'))

**Step Eight** Dropping the 'Cleaned_Categories' column.

In [ ]:
ddf = ddf.drop(columns=['Cleaned_Categories'])

**Step Nine** Getting the cleaned result and verifying the information.

In [ ]:
with ProgressBar():
    result_dask = ddf.compute()

In [ ]:
result_dask.head(50)

In [ ]:
result_dask.info()

**Step Ten** Function to sort the data into periods. Pre-pandemic 2018-2019. Pandemic 2020-2021, and Post-Pandemic 2022-2023. 

**Note** Years can be changed if you want to look at different years.

In [ ]:
def assign_period(year):
    if 2018 <= year <= 2019:
        return 'Pre_Pandemic'
    elif 2020 <= year <= 2021: 
        return 'Pandemic'
    elif 2022 <= year <= 2023:
        return 'Post_Pandemic'
    else:
        return 'Undefined'

**Optional** Converting data back to dask to handle the assignment of periods. 

**Note** Can also be done with Pandas if your computer can handle large datasets.

In [ ]:
ddf = dd.from_pandas(result_dask, npartitions=8) if isinstance(result_dask, pd.DataFrame) else result_dask

**Step Eleven** Adding the new 'Period' column using the function and the dask dataframe.

In [ ]:
ddf['Period'] = ddf['CheckoutYear'].apply(assign_period, meta=('x', 'O'))

**Step Twelve** Getting the results and converting back to a pandas dataframe.

In [ ]:
with ProgressBar():
    result_dask_updated = ddf.compute()

In [ ]:
result_dask_updated.head(50)

**Optional** Another conversion of the 'Cleaned_Year' column to Int64 datatype if you missed doing it earlier.

In [ ]:
result_dask_updated['Cleaned_Year'] = result_dask_updated['Cleaned_Year'].astype('Int64')

**Step Thirteen** Saving the final cleaned file.

In [ ]:
result_dask_updated.to_csv('cleanedv2_checkouts_data.csv', index=False)

**End Comments** This file can now be loaded into Tableau for dashboarding purposes.